ens_exposed[0]# Comparison of scenarios

1. re

In [1]:
import copy
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx
from ItalySetup import ItalySetup
from covidOCP import COVIDVaccinationOCP, COVIDParametersOCP
from main import pick_scenario, build_scenario
import seaborn as sns
import datetime
import pickle
import os

nx = 9
states_names = ['S', 'E', 'P', 'I', 'A', 'Q', 'H', 'R', 'V']
when = 'future'
file_prefix = f'week'
outdir = '2021-01-19-107_90/'

n_int_steps = 6
nnodes = 107  # nodes
ndays_ocp = 90
ndays = 180

setup = ItalySetup(nnodes, ndays, when)
setup_ocp = ItalySetup(nnodes, ndays_ocp, when)
M = setup.nnodes
N = len(setup.model_days) - 1

with open(f'italy-data/parameters_{nnodes}_{when}.pkl', 'rb') as inp:
    p = pickle.load(inp)
    
os.makedirs('model_output/scenarios', exist_ok=True)

scenarios = {pick_scenario(setup, i)['name']:pick_scenario(setup, i) for i in np.arange(30)}
scenarios.keys()

Loaded Italy Setup with 107 nodes.
Loaded Italy Setup with 107 nodes.


dict_keys(['U-r1-t125000-id0', 'L-r1-t125000-id1', 'U-r2-t125000-id2', 'L-r2-t125000-id3', 'U-r15-t125000-id4', 'L-r15-t125000-id5', 'U-r1-t250000-id6', 'L-r1-t250000-id7', 'U-r2-t250000-id8', 'L-r2-t250000-id9', 'U-r15-t250000-id10', 'L-r15-t250000-id11', 'U-r1-t479700-id12', 'L-r1-t479700-id13', 'U-r2-t479700-id14', 'L-r2-t479700-id15', 'U-r15-t479700-id16', 'L-r15-t479700-id17', 'U-r1-t1000000-id18', 'L-r1-t1000000-id19', 'U-r2-t1000000-id20', 'L-r2-t1000000-id21', 'U-r15-t1000000-id22', 'L-r15-t1000000-id23', 'U-r1-t2000000-id24', 'L-r1-t2000000-id25', 'U-r2-t2000000-id26', 'L-r2-t2000000-id27', 'U-r15-t2000000-id28', 'L-r15-t2000000-id29'])

In [28]:
# Choose a subset of scenarios:
pick = 'L-r15'
scenarios = {k:v for (k,v) in scenarios.items() if pick in k}
print(len(scenarios))

5


import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())



beta_scenarios = ['U', 'L']   # thankfully 0 and 1 are U and L respecetively. But ugly

all_sims = {}


def runall(dr, scenario_mu):
    p = COVIDParametersOCP.OCParameters(setup=setup, M=M, when=when, posterior_draw=dr+1)
    p.apply_epicourse(setup, scenario_mu)
    control_initial = np.zeros((M, N))
    results, state_initial, yell, mob = COVIDVaccinationOCP.integrate(N,
                                                                      setup=setup,
                                                                      parameters=p,
                                                                      controls=control_initial,
                                                                      save_to=f'{outdir}{prefix}-int-{nnodes}_{ndays}-nc',
                                                                      method='rk4',
                                                                      n_rk4_steps=n_int_steps)
    return results

for i, beta_scn in enumerate(beta_scenarios):
    all_sims[beta_scn]= []

    scenario = pick_scenario(setup=setup, scn_id=i)
    
    all_sims[beta_scn] = pool.starmap(runall, [(dr, scenario['beta_mult']) for dr in range(3)])

In [30]:
scenarios_opt = {}
scenarios_baseline = {}
## Re-integrate vacc
for scenario_name, scenario in scenarios.items():
    fname = f"{outdir}{file_prefix}-{'-'.join(scenario_name.split('-')[:-1])}-opt-{nnodes}_{ndays_ocp}.csv"
    try:
        md = pd.read_csv(fname, index_col= 'date', parse_dates=True)
        print(f'YES {fname}')
    
        # Build scenario
        maxvaccrate_regional, delivery_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
        M = setup.nnodes
        N = setup.ndays - 1
        control_initial = np.zeros((M, N))
        unvac_nd = np.copy(setup.pop_node)
        stockpile = 0
        for k in range(ndays_ocp - 1):
            stockpile += delivery_national[k]
            for nodename in md.place.unique():
                nd = setup.ind2name.index(nodename)
                to_allocate = md[(md['place'] == nodename) & (md['comp'] == 'vacc')].iloc[k]['value']
                to_allocate = min(to_allocate, maxvaccrate_regional[nd, k], unvac_nd[nd], stockpile)
                control_initial[nd, k] = to_allocate
                stockpile -= to_allocate
                unvac_nd[nd] -= to_allocate
        p.apply_epicourse(setup, scenario['beta_mult'])
        # END Build scenario

        results, state_initial, yell, mob = COVIDVaccinationOCP.integrate(N,
                                                                          setup=setup,
                                                                          parameters=p,
                                                                          controls=control_initial,
                                                                          save_to=f'model_output/scenarios/{scenario_name}-opi-{nnodes}_{ndays}',
                                                                          n_rk4_steps=n_int_steps)
        results.set_index('date', drop=True, inplace=True)
        scenarios_opt[scenario_name] = scenario
        print(f'--> DONE {scenario_name}')

        if scenario_name.split('-')[0] not in scenarios_baseline:
            control_initial = np.zeros((M, N))
            # Generate NO vaccination scenarios
            results, state_initial, yell, mob = COVIDVaccinationOCP.integrate(N,
                                                                      setup=setup,
                                                                      parameters=p,
                                                                      controls=control_initial,
                                                                      save_to=f'model_output/scenarios/{scenario_name}-novacc-{nnodes}_{ndays}',
                                                                      n_rk4_steps=n_int_steps)
            scenarios_baseline[scenario_name.split('-')[0]] = pd.read_csv(f'model_output/scenarios/{scenario_name}-novacc-{nnodes}_{ndays}.csv', 
                                                                         index_col= 'date', parse_dates=True)


    except FileNotFoundError:
        print(f'NOT {fname}')

YES 2021-01-19-107_90/week-U-r1-t125000-opt-107_90.csv


  1%|          | 2/179 [00:00<00:09, 19.64it/s]

===> Integrating for model_output/scenarios/U-r1-t125000-id0-opi-107_180


  3%|▎         | 5/179 [00:00<00:08, 20.24it/s]

--> DONE U-r1-t125000-id0
===> Integrating for model_output/scenarios/U-r1-t125000-id0-novacc-107_180


...

--> DONE U-r15-t2000000-id28
YES 2021-01-19-107_90/week-L-r15-t2000000-opt-107_90.csv


  1%|          | 1/179 [00:00<00:19,  9.06it/s]

===> Integrating for model_output/scenarios/L-r15-t2000000-id29-opi-107_180


100%|██████████| 179/179 [00:15<00:00, 11.23it/s]


--> DONE L-r15-t2000000-id29


In [31]:
# Generate alternatives:
# BY INCIDENCE: 
for scenario_name, scenario in scenarios_opt.items():
    nv = scenarios_baseline[scenario_name.split('-')[0]]
    incid = nv[nv['comp'].isin(['E', 'P', 'I', 'A', 'Q', 'H', 'R'])].groupby('placeID').sum()
    incid.sort_values('value', ascending=False)
    
    # Build scenario
    maxvaccrate_regional, delivery_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
    M = setup.nnodes
    N = setup.ndays - 1
    control_initial = np.zeros((M, N))
    unvac_nd = np.copy(setup.pop_node)
    stockpile = 0
    for k in range(ndays_ocp -1):
        stockpile += delivery_national[k]
        for nodename in md.place.unique():
            nd = setup.ind2name.index(nodename)
            to_allocate = stockpile * incid.loc[nd]['value'] / incid['value'].sum()
            to_allocate = min(to_allocate, maxvaccrate_regional[nd, k], unvac_nd[nd], stockpile)
            control_initial[nd, k] = to_allocate
            stockpile -= to_allocate
            unvac_nd[nd] -= to_allocate
    p.apply_epicourse(setup, scenario['beta_mult'])
    # END Build scenario

    results, state_initial, yell, mob = COVIDVaccinationOCP.integrate(N,
                                                                      setup=setup,
                                                                      parameters=p,
                                                                      controls=control_initial,
                                                                      save_to=f'model_output/scenarios/{scenario_name}-inc-{nnodes}_{ndays}',
                                                                      n_rk4_steps=n_int_steps)
    results.set_index('date', drop=True, inplace=True)

# BY Suceptibility: 
for scenario_name, scenario in scenarios_opt.items():
    nv = scenarios_baseline[scenario_name.split('-')[0]]
    incid = nv[nv['comp'].isin(['S'])].loc[str(setup.start_date)]
    incid.set_index('placeID', inplace=True)
    
    # Build scenario
    maxvaccrate_regional, delivery_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
    M = setup.nnodes
    N = setup.ndays - 1
    control_initial = np.zeros((M, N))
    unvac_nd = np.copy(setup.pop_node)
    stockpile = 0
    for k in range(ndays_ocp -1):
        stockpile += delivery_national[k]
        for nodename in md.place.unique():
            nd = setup.ind2name.index(nodename)
            to_allocate = stockpile * incid.loc[nd]['value'] / incid['value'].sum()
            to_allocate = min(to_allocate, maxvaccrate_regional[nd, k], unvac_nd[nd], stockpile)
            control_initial[nd, k] = to_allocate
            stockpile -= to_allocate
            unvac_nd[nd] -= to_allocate
    p.apply_epicourse(setup, scenario['beta_mult'])
    # END Build scenario

    results, state_initial, yell, mob = COVIDVaccinationOCP.integrate(N,
                                                                      setup=setup,
                                                                      parameters=p,
                                                                      controls=control_initial,
                                                                      save_to=f'model_output/scenarios/{scenario_name}-sus-{nnodes}_{ndays}',
                                                                      n_rk4_steps=n_int_steps)
    results.set_index('date', drop=True, inplace=True)
    

    
# BY POPULATION 
for scenario_name, scenario in scenarios_opt.items():
    # Build scenario
    maxvaccrate_regional, delivery_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
    M = setup.nnodes
    N = setup.ndays - 1
    control_initial = np.zeros((M, N))
    unvac_nd = np.copy(setup.pop_node)
    stockpile = 0
    for k in range(ndays_ocp -1):
        stockpile += delivery_national[k]
        for nodename in md.place.unique():
            pop_nd = setup.pop_node[nd]
            nd = setup.ind2name.index(nodename)
            to_allocate = stockpile * pop_nd / setup.pop_node.sum()
            to_allocate = min(to_allocate, maxvaccrate_regional[nd, k], unvac_nd[nd], stockpile)
            control_initial[nd, k] = to_allocate
            stockpile -= to_allocate
            unvac_nd[nd] -= to_allocate
    p.apply_epicourse(setup, scenario['beta_mult'])
    # END Build scenario

    results, state_initial, yell, mob = COVIDVaccinationOCP.integrate(N,
                                                                      setup=setup,
                                                                      parameters=p,
                                                                      controls=control_initial,
                                                                      save_to=f'model_output/scenarios/{scenario_name}-pop-{nnodes}_{ndays}',
                                                                      n_rk4_steps=n_int_steps)
    results.set_index('date', drop=True, inplace=True)
    
# BY FUTURE NEW CASES
for scenario_name, scenario in scenarios_opt.items():
    nv = scenarios_baseline[scenario_name.split('-')[0]]
    incid = nv[nv['comp'].isin(['E', 'I', 'A'])].groupby('placeID').sum()
    incid.sort_values('value', ascending=False)
    
    # Build scenario
    maxvaccrate_regional, delivery_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
    M = setup.nnodes
    N = setup.ndays - 1
    control_initial = np.zeros((M, N))
    unvac_nd = np.copy(setup.pop_node)
    stockpile = 0
    for k in range(ndays_ocp - 1):
        stockpile += delivery_national[k]
        for nodename in md.place.unique():
            nd = setup.ind2name.index(nodename)
            to_allocate = stockpile * incid.loc[nd]['value'] / incid['value'].sum()
            to_allocate = min(to_allocate, maxvaccrate_regional[nd, k], unvac_nd[nd], stockpile)
            control_initial[nd, k] = to_allocate
            stockpile -= to_allocate
            unvac_nd[nd] -= to_allocate
    p.apply_epicourse(setup, scenario['beta_mult'])
    # END Build scenario

    results, state_initial, yell, mob = COVIDVaccinationOCP.integrate(N,
                                                                      setup=setup,
                                                                      parameters=p,
                                                                      controls=control_initial,
                                                                      save_to=f'model_output/scenarios/{scenario_name}-cas-{nnodes}_{ndays}',
                                                                      n_rk4_steps=n_int_steps)
    results.set_index('date', drop=True, inplace=True)

#TODO: Centrality based and R0 based

  1%|          | 2/179 [00:00<00:13, 12.76it/s]

===> Integrating for model_output/scenarios/U-r1-t125000-id0-inc-107_180


  2%|▏         | 4/179 [00:00<00:11, 15.02it/s]

===> Integrating for model_output/scenarios/L-r1-t125000-id1-inc-107_180


  1%|          | 2/179 [00:00<00:12, 14.36it/s]

...

  2%|▏         | 3/179 [00:00<00:09, 18.48it/s]

===> Integrating for model_output/scenarios/U-r15-t2000000-id28-cas-107_180


  2%|▏         | 3/179 [00:00<00:09, 18.68it/s]

===> Integrating for model_output/scenarios/L-r15-t2000000-id29-cas-107_180


100%|██████████| 179/179 [00:08<00:00, 20.81it/s]


In [2]:
import matlab.engine

eng = matlab.engine.start_matlab()
eng.cd('check-strats/', nargout=0)

In [3]:
scenarios = {pick_scenario(setup, i)['name']:pick_scenario(setup, i) for i in np.arange(30)}
scenarios.keys()

dict_keys(['U-r1-t125000-id0', 'L-r1-t125000-id1', 'U-r2-t125000-id2', 'L-r2-t125000-id3', 'U-r15-t125000-id4', 'L-r15-t125000-id5', 'U-r1-t250000-id6', 'L-r1-t250000-id7', 'U-r2-t250000-id8', 'L-r2-t250000-id9', 'U-r15-t250000-id10', 'L-r15-t250000-id11', 'U-r1-t479700-id12', 'L-r1-t479700-id13', 'U-r2-t479700-id14', 'L-r2-t479700-id15', 'U-r15-t479700-id16', 'L-r15-t479700-id17', 'U-r1-t1000000-id18', 'L-r1-t1000000-id19', 'U-r2-t1000000-id20', 'L-r2-t1000000-id21', 'U-r15-t1000000-id22', 'L-r15-t1000000-id23', 'U-r1-t2000000-id24', 'L-r1-t2000000-id25', 'U-r2-t2000000-id26', 'L-r2-t2000000-id27', 'U-r15-t2000000-id28', 'L-r15-t2000000-id29'])

In [4]:
scn_results = pd.DataFrame(columns=['newdoseperweek', 'method', 'infected'])
methods = ['opi','cas', 'pop', 'sus'] # 'opt'

for met in methods:
    for scenario_name, scenario in scenarios.items():
        filename = f'model_output/scenarios/{scenario_name}-{met}-{nnodes}_{ndays}.csv'
        if os.path.isfile(filename):
            print(f'Doing {filename}')
            maxvaccrate_regional, stockpile_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
            p.apply_epicourse(setup, scenario['beta_mult'])
            
            md = pd.read_csv(filename, index_col= 'date', parse_dates=True)
            
            dosesV = md[md['comp'] == 'vacc'].pivot(columns='placeID', values='value').to_numpy()

            
            eng.workspace['Vdoses'] = matlab.double(dosesV.tolist())
            eng.workspace['beta_ratio'] = matlab.double(scenario['beta_mult'][0].tolist())
            eng.run('main_script.m', nargout=0)
            ens_exposed = np.array(eng.eval('ens_exposed_preprocess')).flatten()
        
            scn_results = pd.concat([scn_results, pd.DataFrame.from_dict({'newdoseperweek':[int(scenario_name.split('-')[2][1:])]*len(ens_exposed),
                                                                           'method': [met]*len(ens_exposed),
                                                                           'infected':ens_exposed.tolist(),
                                                                           'post_sample':np.arange(len(ens_exposed)),
                                                                           'doses': [dosesV.sum()]*len(ens_exposed),
                                                                           'scenario-beta': [scenario_name.split('-')[0]]*len(ens_exposed),
                                                                           'scenario-rate': [scenario_name.split('-')[1]]*len(ens_exposed),
                                                                           'scenario-tot': [scenario_name.split('-')[2]]*len(ens_exposed),
                                                                           'scenario': [scenario_name]*len(ens_exposed)
                                                                         })])
        else:
            print(f'not found {scenario_name}')

Doing model_output/scenarios/U-r1-t125000-id0-opi-107_180.csv
Doing model_output/scenarios/L-r1-t125000-id1-opi-107_180.csv
Doing model_output/scenarios/U-r2-t125000-id2-opi-107_180.csv
Doing model_output/scenarios/L-r2-t125000-id3-opi-107_180.csv
Doing model_output/scenarios/U-r15-t125000-id4-opi-107_180.csv
Doing model_output/scenarios/L-r15-t125000-id5-opi-107_180.csv
Doing model_output/scenarios/U-r1-t250000-id6-opi-107_180.csv
Doing model_output/scenarios/L-r1-t250000-id7-opi-107_180.csv
Doing model_output/scenarios/U-r2-t250000-id8-opi-107_180.csv
Doing model_output/scenarios/L-r2-t250000-id9-opi-107_180.csv
Doing model_output/scenarios/U-r15-t250000-id10-opi-107_180.csv
Doing model_output/scenarios/L-r15-t250000-id11-opi-107_180.csv
Doing model_output/scenarios/U-r1-t479700-id12-opi-107_180.csv
Doing model_output/scenarios/L-r1-t479700-id13-opi-107_180.csv
Doing model_output/scenarios/U-r2-t479700-id14-opi-107_180.csv
Doing model_output/scenarios/L-r2-t479700-id15-opi-107_180.cs

In [10]:
scenarios_baseline = {}
for scenario_name, scenario in scenarios.items():
    if scenario_name.split('-')[0] not in scenarios_baseline:
        print(scenario_name)

        maxvaccrate_regional, stockpile_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
        
        p.apply_epicourse(setup, scenario['beta_mult'])


        dosesV = np.zeros_like(dosesV)


        eng.workspace['Vdoses'] = matlab.double(dosesV.tolist())
        eng.workspace['beta_ratio'] = matlab.double(scenario['beta_mult'][0].tolist())
        eng.run('main_script.m', nargout=0)
        ens_exposed = np.array(eng.eval('ens_exposed_preprocess')).flatten()
        
        scenarios_baseline[scenario_name.split('-')[0]] = ens_exposed
        

U-r1-t125000-id0
L-r1-t125000-id1


In [34]:
scn_results['bl'] = 0
scn_results.loc[scn_results['scenario-beta'] == 'U', 'bl'] = np.tile(scenarios_baseline['U'],60)
scn_results.loc[scn_results['scenario-beta'] == 'L', 'bl'] = np.tile(scenarios_baseline['L'],60)

In [41]:
scn_results['averted'] = scn_results['bl'] - scn_results['infected']
scn_results['avertedpervacc'] = scn_results['averted']/scn_results['doses']

In [42]:
scn_results.to_csv('model_output/scenarios/all_summary.csv')

In [40]:
scn_results

,newdoseperweek,method,infected,post_sample,doses,scenario-beta,scenario-rate,scenario-tot,scenario,bl,averted
0,125000,opi,6.004858e+08,0.0,1.566664e+06,U,r1,t125000,U-r1-t125000-id0,6.101198e+08,9.634039e+06
1,125000,opi,1.025928e+09,1.0,1.566664e+06,U,r1,t125000,U-r1-t125000-id0,1.044931e+09,1.900305e+07
2,125000,opi,6.361166e+08,2.0,1.566664e+06,U,r1,t125000,U-r1-t125000-id0,6.448196e+08,8.702985e+06
3,125000,opi,1.076014e+09,3.0,1.566664e+06,U,r1,t125000,U-r1-t125000-id0,1.094167e+09,1.815359e+07
4,125000,opi,6.857489e+08,4.0,1.566664e+06,U,r1,t125000,U-r1-t125000-id0,6.927140e+08,6.965141e+06
...,...,...,...,...,...,...,...,...,...,...,...
95,2000000,sus,1.316082e+08,95.0,2.499106e+07,L,r15,t2000000,L-r15-t2000000-id29,1.381956e+08,6.587377e+06
96,2000000,sus,1.126511e+08,96.0,2.499106e+07,L,r15,t2000000,L-r15-t2000000-id29,1.173939e+08,4.742858e+06
97,2000000,sus,1.412437e+08,97.0,2.499106e+07,L,r15,t2000000,L-r15-t2000000-id29,1.485987e+08,7.354985e+06
98,2000000,sus,1.321285e+08,98.0,2.499106e+07,L,r15,t2000000,L-r15-t2000000-id29,1.397171e+08,7.588535e+06
